Playing with 02_Andes_2000_2019_CR2_figura_2023 file from Caro 

In [1]:
##########################################################################
# global initilization & pre-processed tasks #############################

# python module import ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
from oggm import cfg, utils, workflow, tasks, graphics
import matplotlib.pyplot as plt
import logging
import oggm
from oggm import cfg, utils, workflow, tasks, graphics, global_tasks
from oggm.core import massbalance, flowline
from oggm.shop import gcm_climate
from oggm.core.massbalance import ScalarMassBalance
from oggm.tests.funcs import bu_tidewater_bed
from oggm.core.flowline import FluxBasedModel

import pandas as pd
from oggm.core.massbalance import MultipleFlowlineMassBalance # mass-balance at the glacier level

import xarray as xr
import numpy as np
import pandas as pd
import os
import geopandas as gpd

In [2]:
cfg.initialize(logging_level='WARNING')
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['continue_on_error'] = True
cfg.PARAMS['run_mb_calibration'] = True
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['border'] = 80
cfg.PARAMS['store_fl_diagnostics'] = True

cfg.PARAMS["climate_qc_months"] = 3 # hugonnet 
cfg.PARAMS["hydro_month_nh"] = 1 # hugonnet
cfg.PARAMS["hydro_month_sh"] = 1 # hugonnet
cfg.PARAMS["max_mu_star"] = 600

cfg.PARAMS["temp_all_solid"] = 0.0
cfg.PARAMS["temp_all_liq"] = 2.0 
cfg.PARAMS["temp_melt"] = 0

2025-05-26 11:02:12: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2025-05-26 11:02:12: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2025-05-26 11:02:12: oggm.cfg: Multiprocessing: using all available processors (N=8)
2025-05-26 11:02:14: oggm.cfg: Multiprocessing switched ON after user settings.
2025-05-26 11:02:14: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2025-05-26 11:02:14: oggm.cfg: WARNING: adding an unknown parameter `run_mb_calibration`:`True` to PARAMS.
2025-05-26 11:02:14: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2025-05-26 11:02:14: oggm.cfg: PARAMS['store_fl_diagnostics'] changed from `False` to `True`.
2025-05-26 11:02:14: oggm.cfg: WARNING: adding an unknown parameter `climate_qc_months`:`3` to PARAMS.
2025-05-26 11:02:14: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2025-05-26 11:02:14: oggm.cfg: PARAMS['hydro_month_sh'] changed fr

In [3]:
# # glacier call #############################
# datos_rgi = pd.read_csv('/home/caro/04_Andes_2000_2019_CR2/datos/RGI_BNA_Clusters.csv')
# # call parameters
# datos_param = pd.read_csv('/home/caro/04_Andes_2000_2019_CR2/datos/LR_Pf.csv')

# glacier call #############################
datos_rgi = pd.read_csv('/Users/milliespencer/Desktop/files_chile_OGGM_climate_comparison/cautin_RGI_BNA_Clusters.csv')
# call parameters
datos_param = pd.read_csv('/Users/milliespencer/Desktop/files_chile_OGGM_climate_comparison/LR_Pf.csv')

In [4]:
# # INICIO ITERACION por region GLACIO +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# listas de llamado
list_region = ['OT3','DA1','DA2','DA3','WA1','WA2','WA3','WA4','WA5','WA6']


for zona in list_region:
    
# directorio
 #   zona = 'OT3'
    salida = '/Users/milliespencer/Desktop/files_chile_OGGM_climate_comparison' + zona + '/'
    cfg.PATHS['working_dir'] = salida                                 
    compilacion_zona = salida + 'run_output_2000_2019_hydro_TC_' + zona + '.nc'

# extraccion de glaciares por zona
    gdf_sel = datos_rgi[datos_rgi['Cluster'] == zona]
    gdf_sel = list(gdf_sel.RGIId)
    
# se extrae LT usando id glaciar
    param = datos_param[datos_param['Cluster'] == zona]
    LT_n = float(param.LR)
# se extrae Pf 
    Pf_n = float(param.Pf)

    # definion LT en modelo
    cfg.PARAMS["temp_default_gradient"] = LT_n
    cfg.PARAMS["prcp_scaling_factor"] = Pf_n

# entradas iniciales     
    zonax = xr.open_dataset(compilacion_zona)
    rgi_id = zonax['rgi_id'].values
    area = zonax['area'].values

# area total por clusters desde RGI 6

    rgi_area = datos_rgi[datos_rgi['Cluster'] == zona]
    rgi_area = rgi_area.Area.sum()
    
# llamado de simulaciones por zonas glacio, 13 en total

# zona = 'OT3' 
# directorio = '/home/caro/03_Andes_2000_2019_TC/'
# llamado_zona = directorio + zona + '/'
# compilacion_zona = llamado_zona + 'run_output_2000_2019_hydro_TC_' + zona + '.nc'

# zonax = xr.open_dataset(compilacion_zona)
# rgi_id = zonax['rgi_id'].values
# area = zonax['area'].values


    # calculo area usando simulacion
            # revisar numero de filas, deben ser 21.
    
    ID_area = pd.DataFrame(area)
    ID_rgi = pd.DataFrame(rgi_id)
    ID_rgi.rename(columns={ID_rgi.columns[0]:'id' }, inplace = True)
    
    # ver desde que año comienza sim area,
    # debería ser 2000, pero algoa pasa
    ID_area = pd.DataFrame(ID_area.iloc[10])#.transpose() # sacar area
    
    ID_area.rename(columns={ID_area.columns[0]:'area' }, inplace = True)
    ID_area = pd.concat([ID_rgi.reset_index(drop=True), ID_area], axis=1) # merge two dataframes
    ID_area = ID_area.dropna(axis=0) # delere na rows
    max_n = ID_area[ID_area.columns[0]].count()
    ID_area['number'] = range(0,max_n)
    ID_data = ID_area.number.values.tolist()
    ID_rgi_id = ID_area.id.values.tolist()
    
    #     # calculo area usando RGI
        
    # ID_rgi = pd.DataFrame(rgi_id)
    # ID_rgi.rename(columns={ID_rgi.columns[0]:'id' }, inplace = True)
    
    # # llamo hugonnet areas mismas de rgi
    # gmb = utils.get_geodetic_mb_dataframe()
    # gmb.to_csv('/home/caro/OGGM_Maipo_terraclimate_2_2000_2100/gmb_hugonnet.csv')
    # gmb = gmb.loc[gmb['period'] == '2000-01-01_2020-01-01'] # extraigo solo '2000-01-01_2020-01-01'
    # gmb['rgiid2'] = gmb.index 
    # gmb = gmb[gmb['rgiid2'].isin(ID_rgi['id'] )]
    
    # ID_area = pd.DataFrame(gmb['area'])
    # ID_area['rgi_id'] = ID_area.index 
    
    # max_n = ID_area[ID_area.columns[0]].count()
    # ID_area['number'] = range(0,max_n)
    # ID_data = ID_area.number.values.tolist()
    # ID_rgi_id = ID_area.rgi_id.values.tolist()

/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_16158/3031429275.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  LT_n = float(param.LR)
/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_16158/3031429275.py:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Pf_n = float(param.Pf)
2025-05-26 11:04:21: oggm.cfg: WARNING: adding an unknown parameter `prcp_scaling_factor`:`1.0` to PARAMS.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/milliespencer/Desktop/files_chile_OGGM_climate_comparisonOT3/run_output_2000_2019_hydro_TC_OT3.nc'